In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Data = pd.read_csv("../input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
def pre_proc():
    df = Data.copy()

In [ ]:
df = Data.copy()

In [ ]:
df

In [ ]:
df["RainTomorrow"].value_counts()

major = df[df["RainTomorrow"] == "No"]
minor = df[df["RainTomorrow"] == "Yes"]

major = major.sample(frac=1)

major = major[:31877]

df = pd.concat([major,minor],axis=0)

df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
df["WindGustDir"] = le.fit_transform(df["WindGustDir"])
df["WindDir9am"] = le.fit_transform(df["WindDir9am"])
df["WindDir3pm"] = le.fit_transform(df["WindDir3pm"])
df["Location"] = le.fit_transform(df["Location"])

In [ ]:
df = df.drop(["Date"],axis=1)

In [ ]:
df = df.fillna("0")

In [ ]:
df.isna().sum()

In [ ]:
df = df.replace('0',0)
df = df.replace('Yes',1)
df = df.replace('No',0)

In [ ]:
df = df.astype(float)
df.dtypes

In [ ]:
y = df["RainTomorrow"]
X = df.drop(["RainTomorrow"],axis=1)

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=42)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 16))
sns.heatmap(df.corr(), cmap="hot")

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(20, 2), random_state=42)

clf.fit(X_train, y_train)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
val_pred = clf.predict(X_train)
print(classification_report(y_train, val_pred)) # not overfit

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adb_classifier = AdaBoostClassifier(DecisionTreeClassifier(criterion = 'entropy', random_state = 42),
                                    n_estimators=2000,
                                    learning_rate=0.1,
                                    algorithm='SAMME.R',
                                    random_state=42,)
adb_classifier.fit(X_train, y_train)
y_pred_adb = adb_classifier.predict(X_test)
accuracy_score(y_test, y_pred_adb)